In [1]:
from transformations import Compose, DenseTarget
from transformations import MoveAxis, Normalize01
from customdatasets2 import SegmentationDataSet
from torch.utils.data import DataLoader
from customdatasets import make_mask
from skimage.io import imread
from sklearn.model_selection import train_test_split
import pathlib
import numpy as np
import pandas as pd

df = pd.read_csv("understanding_cloud_organization/train_320.csv")

# root directory
root = pathlib.Path.cwd() / 'understanding_cloud_organization'
def get_filenames_of_path(path: pathlib.Path, ext: str = '*'):
    """Returns a list of files in a directory/path. Uses pathlib."""
    filenames = [file for file in path.glob(ext) if file.is_file()]
    return filenames

# input and target files
inputs = get_filenames_of_path(root / 'train_images_320')  
true_inputs = []
true_targets = []

shape = (320, 480)


for input in inputs:
    img_name = str(input).split('/')[-1]
    array_input = imread(input)
    true_inputs.append(array_input)
    true_inputs.append(np.fliplr(array_input))
    true_inputs.append(np.flipud(array_input))
    mask = make_mask(df, img_name, shape)
    true_targets.append(mask)
    true_targets.append(np.fliplr(mask))
    true_targets.append(np.flipud(mask))
    
# training transformations and augmentations
transforms = Compose([
    DenseTarget(),
    Normalize01(),
    MoveAxis(),
])

# random seed
random_seed = 42

# split dataset into training set and validation set
train_size = 0.8  # 80:20 split

inputs_train, inputs_valid = train_test_split(
    true_inputs,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)


targets_train, targets_valid = train_test_split(
    true_targets,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)

# dataset training
dataset_train = SegmentationDataSet(inputs=inputs_train,
                                    targets=targets_train,
                                    transform=transforms)

    

# dataset validation
dataset_valid = SegmentationDataSet(inputs=inputs_valid,
                                    targets=targets_valid,
                                    transform=transforms)


# dataloader training
dataloader_training = DataLoader(dataset=dataset_train,
                                 batch_size=4,
                                 shuffle=False)
# dataloader validation
dataloader_validation = DataLoader(dataset=dataset_valid,
                                   batch_size=4,
                                   shuffle=False)

dataloaders = {
    'train': dataloader_training,
    'val': dataloader_validation
} 




/usr/lib/python3/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [1]:
import pandas as pd
sample_submission = pd.read_csv("understanding_cloud_organization/sample_submission.csv")
train = pd.read_csv("understanding_cloud_organization/train_320.csv")

In [2]:
train['Image_name'] = train['Image_Label'].apply(lambda x: x.split('_')[0])
train['Label_name'] = train['Image_Label'].apply(lambda x: x.split('_')[1])
train.drop('Image_Label',axis=1,inplace=True)
train = train.pivot('Image_name','Label_name','EncodedPixels')

In [3]:
train.head()

Label_name,Fish,Flower,Gravel,Sugar
Image_name,,,,
0011165.jpg,14154 214 14474 214 14794 214 15114 214 15434 ...,71125 229 71445 229 71765 229 72085 229 72405 ...,NaN,NaN
002be4f.jpg,12484 201 12804 201 13124 201 13444 201 13764 ...,70282 119 70602 119 70922 119 71242 119 71562 ...,NaN,3589 80 3909 80 4229 80 4549 80 4869 80 5189 8...
0031ae9.jpg,484 157 804 157 1124 157 1444 157 1764 157 208...,469 161 789 161 1109 161 1429 161 1749 161 203...,NaN,34600 89 34920 89 35240 89 35560 89 35880 89 3...
0035239.jpg,NaN,5444 106 5764 106 6084 106 6404 106 6724 106 7...,3750 87 4070 87 4390 87 4710 87 5030 87 5350 8...,NaN
003994e.jpg,123970 3 123975 1 123977 16 124282 1 124285 3 ...,NaN,18679 96 18999 96 19319 96 19639 96 19959 96 2...,1604 112 1924 112 2244 112 2564 112 2884 112 3...


In [9]:
import os
path = 'understanding_cloud_organization'
n_train = len(os.listdir(f'{path}/train_images_350'))
n_test = len(os.listdir(f'{path}/test_images_350'))
print(f'There are {n_train} images in train dataset')
print(f'There are {n_test} images in test dataset')

There are 5547 images in train dataset
There are 1987 images in test dataset


In [13]:
train['Image_name'].apply(lambda x: x.split('_')[1]).value_counts()

KeyError: 'Image_name'

In [2]:
import torch
from resnet_unet import ResNetUNet 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNetUNet(n_class=4)
model = model.to(device)

# check keras-like model summary using torchsummary
from torchsummary import summary
summary(model, input_size=(3, 320, 480))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 320, 480]           1,792
              ReLU-2         [-1, 64, 320, 480]               0
            Conv2d-3         [-1, 64, 320, 480]          36,928
              ReLU-4         [-1, 64, 320, 480]               0
            Conv2d-5         [-1, 64, 160, 240]           9,408
            Conv2d-6         [-1, 64, 160, 240]           9,408
       BatchNorm2d-7         [-1, 64, 160, 240]             128
       BatchNorm2d-8         [-1, 64, 160, 240]             128
              ReLU-9         [-1, 64, 160, 240]               0
             ReLU-10         [-1, 64, 160, 240]               0
        MaxPool2d-11          [-1, 64, 80, 120]               0
        MaxPool2d-12          [-1, 64, 80, 120]               0
           Conv2d-13          [-1, 64, 80, 120]          36,864
           Conv2d-14          [-1, 64, 

In [3]:
from collections import defaultdict
import torch.nn.functional as F
from loss import dice_loss

def calc_loss(pred, target, metrics, bce_weight=1):
    bce = F.binary_cross_entropy_with_logits(pred, target)

    pred = F.sigmoid(pred)
    dice = dice_loss(pred, target)

    loss = bce * bce_weight + dice * (1 - bce_weight)

    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)

    return loss

def print_metrics(metrics, epoch_samples, phase):
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))

    print("{}: {}".format(phase, ", ".join(outputs)))

def train_model(model, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])

                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = calc_loss(outputs, labels, metrics)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                print("saving best model")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

num_class = 4
model = ResNetUNet(num_class).to(device)

# freeze backbone layers
#for l in model.base_layers:
#    for param in l.parameters():
#        param.requires_grad = False

optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=30, gamma=0.1)

model = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=7)

cuda:0
Epoch 0/6
----------
LR 0.0001


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


train: bce: 0.329540, dice: 0.808422, loss: 0.329540
val: bce: 0.289273, dice: 0.767563, loss: 0.289273
saving best model
11m 3s
Epoch 1/6
----------
LR 0.0001


In [ ]:
encoded_pixels = []
loaders = {"infer": valid_loader}
runner.infer(
    model=model,
    loaders=loaders,
    callbacks=[
        CheckpointCallback(
            resume=f"{logdir}/checkpoints/best.pth"),
        InferCallback()
    ],
)
valid_masks = []
probabilities = np.zeros((2220, 350, 525))
for i, (batch, output) in enumerate(tqdm.tqdm(zip(
        valid_dataset, runner.callbacks[0].predictions["logits"]))):
    image, mask = batch
    for m in mask:
        if m.shape != (350, 525):
            m = cv2.resize(m, dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
        valid_masks.append(m)

    for j, probability in enumerate(output):
        if probability.shape != (350, 525):
            probability = cv2.resize(probability, dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
        probabilities[i * 4 + j, :, :] = probability

In [14]:
class_params = {}
for class_id in range(4):
    print(class_id)
    attempts = []
    for t in range(0, 100, 5):
        t /= 100
        for ms in [0, 100, 1200, 5000, 10000]:
            masks = []
            for i in range(class_id, len(probabilities), 4):
                probability = probabilities[i]
                predict, num_predict = post_process(sigmoid(probability), t, ms)
                masks.append(predict)

            d = []
            for i, j in zip(masks, valid_masks[class_id::4]):
                if (i.sum() == 0) & (j.sum() == 0):
                    d.append(1)
                else:
                    d.append(dice(i, j))

            attempts.append((t, ms, np.mean(d)))

    attempts_df = pd.DataFrame(attempts, columns=['threshold', 'size', 'dice'])


    attempts_df = attempts_df.sort_values('dice', ascending=False)
    print(attempts_df.head())
    best_threshold = attempts_df['threshold'].values[0]
    best_size = attempts_df['size'].values[0]
    
    class_params[class_id] = (best_threshold, best_size)

0


NameError: name 'probabilities' is not defined

In [15]:
sns.lineplot(x='threshold', y='dice', hue='size', data=attempts_df);
plt.title('Threshold and min size vs dice for one of the classes');

NameError: name 'sns' is not defined

In [16]:
for i, (input, output) in enumerate(zip(
        valid_dataset, runner.callbacks[0].predictions["logits"])):
    image, mask = input
        
    image_vis = image.transpose(1, 2, 0)
    mask = mask.astype('uint8').transpose(1, 2, 0)
    pr_mask = np.zeros((350, 525, 4))
    for j in range(4):
        probability = cv2.resize(output.transpose(1, 2, 0)[:, :, j], dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
        pr_mask[:, :, j], _ = post_process(sigmoid(probability), class_params[j][0], class_params[j][1])
    #pr_mask = (sigmoid(output) > best_threshold).astype('uint8').transpose(1, 2, 0)
    
        
    visualize_with_raw(image=image_vis, mask=pr_mask, original_image=image_vis, original_mask=mask, raw_image=image_vis, raw_mask=output.transpose(1, 2, 0))
    
    if i >= 2:
        break

NameError: name 'valid_dataset' is not defined

In [17]:
import gc
torch.cuda.empty_cache()
gc.collect()

NameError: name 'torch' is not defined

In [18]:
test_dataset = CloudDataset(df=sub, datatype='test', img_ids=test_ids, transforms = get_validation_augmentation(), preprocessing=get_preprocessing(preprocessing_fn))
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=0)

loaders = {"test": test_loader}

NameError: name 'CloudDataset' is not defined

In [19]:
encoded_pixels = []
image_id = 0
for i, test_batch in enumerate(tqdm.tqdm(loaders['test'])):
    runner_out = runner.predict_batch({"features": test_batch[0].cuda()})['logits']
    for i, batch in enumerate(runner_out):
        for probability in batch:
            
            probability = probability.cpu().detach().numpy()
            if probability.shape != (350, 525):
                probability = cv2.resize(probability, dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
            predict, num_predict = post_process(sigmoid(probability), class_params[image_id % 4][0], class_params[image_id % 4][1])
            if num_predict == 0:
                encoded_pixels.append('')
            else:
                r = mask2rle(predict)
                encoded_pixels.append(r)
            image_id += 1

NameError: name 'tqdm' is not defined

In [20]:
sub['EncodedPixels'] = encoded_pixels
sub.to_csv('submission.csv', columns=['Image_Label', 'EncodedPixels'], index=False)

NameError: name 'sub' is not defined